## Задание 1. Исследовать GAN для генерации точек на параболе

1.1
* Посмотреть что будет если подавать в качестве шума uniform распределение
* Попробовать избавится от линейности на параболе (любыми известными методами)

1.2
* Сделать генерацию фигуры более сложной формы, например круга
* Добится сходимости
* Желательно сделать гиф-анимацию/видео какие точки выдаёт в процессе обучения

1.3
* Предобучить только дискриминатор (сделать его сильным критиком).
* Обучать только генератор (если критик достаточно сильный, то генератор не будет учиться)

In [ ]:
import os
import numpy as np
import math
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from IPython.display import clear_output
from PIL import Image

import torch
import torch.nn as nn
from torch import autograd
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import Dataset, random_split
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.datasets import FashionMNIST
from torchvision.utils import make_grid
from torchvision.utils import save_image

Определяем модели:

In [ ]:
class GenModel(nn.Module):
    def __init__(self, latent_space):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_space, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50,2))

    def forward(self, x):
        return self.model(x)

In [ ]:
class DisModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 25),
            nn.ReLU(),
            nn.Linear(25, 15),
            nn.ReLU(),
            nn.Linear(15,1),
            nn.Sigmoid())
        
    def forward(self, x):
        return self.model(x)

In [ ]:
def get_g_batch(batch_size, latent_dim):
    g_input = torch.randn(size=(batch_size, latent_dim)).cuda()
    #g_input = torch.randn(size=(batch_size, latent_dim)).cuda()
    labels = torch.ones(size=(batch_size,)).cuda()
    return g_input, labels

In [ ]:
def get_mix_batch(batch_size, latent_dim, netG):
    types_of_points = []
    
    # Generate true pairs and true labels
    x = torch.distributions.Uniform(-1, +1).sample((batch_size,)).cuda()
    true_pair = torch.vstack((x, x*x)).T.cuda()
    true_labels = torch.ones(size=(batch_size,)).unsqueeze(1).cuda()
    types_of_points.append(torch.hstack((true_pair, true_labels)))
    
    # Generate fake uniform pairs and fake labels
    if True:
        x_fake = torch.distributions.Uniform(-1, +1).sample((batch_size,)).cuda()
        y_fake = torch.distributions.Uniform(-1, +1).sample((batch_size,)).cuda()
        fake_pair = torch.vstack((x_fake, y_fake)).T
        fake_labels = torch.zeros(size=(batch_size,)).unsqueeze(1).cuda()
        types_of_points.append(torch.hstack((fake_pair, fake_labels)))

    # Generate points from generator and set labels as fake
    if True:
        gan_pair = netG(torch.randn(size=(batch_size, latent_dim)).cuda())
        #gan_pair = netG(torch.rand(size=(batch_size, latent_dim)).cuda())
        gan_labels = torch.zeros(size=(batch_size,)).unsqueeze(1).cuda()
        types_of_points.append(torch.hstack((gan_pair, gan_labels)))
    
    # Stack all types of points
    z = torch.vstack(types_of_points)
    # Shuffle
    z=z[torch.randperm(z.size()[0])]
    
    # Split back to samples and labels
    mixed_pairs = z[:, :2]
    mixed_labels = z[:, 2]
    return mixed_pairs, mixed_labels

In [ ]:
def get_test_loss(model,test_loader,loss_function):
    with torch.no_grad():
        loss_test_total = 0
        for samples, labels in test_loader:
            outputs = model(samples.cuda())
            loss = loss_function(outputs, labels.cuda())
            loss_test_total += loss.item()
        return loss_test_total/len(test_loader)

In [ ]:
def netD_step(netD, batchD, loss_func, optimizer):
    samples, labels = batchD
    optimizer.zero_grad()
    outputs = netD(samples.cuda())
    loss = loss_func(outputs.cuda(), labels.unsqueeze(1).detach().cuda())
    loss.backward()
    optimizer.step()

In [ ]:
def netG_step(netD, netG, batchG, loss_func, optimizer):
    samples, labels = batchG
    optimizer.zero_grad()
    outputs = netD(netG(samples.cuda()))
    loss = loss_func(outputs.cuda(), labels.unsqueeze(1).detach().cuda())
    loss.backward()
    optimizer.step()

In [ ]:
def plot_gen(netG, epoch="Not provided"):
    Gin, _ = get_g_batch(1000, latent_dim)
    out = netG(Gin).cpu()
    plt.scatter(out.detach().numpy()[:, 0], out.detach().numpy()[:, 1], color="blue", s=1)
    plt.title(f'Generator points. End of epoch= {epoch+1}', fontsize=10)
    plt.axis([-1,1,-0.5,1])
    plt.show()

In [ ]:
latent_dim = 5
batch_size = 128
batch_per_epoch = 1000
epochs = 5

netG = GenModel(latent_dim).cuda()
netD = DisModel().cuda()
loss_func = nn.BCELoss().cuda()
optD = torch.optim.Adam(netD.parameters(), lr=0.001)
optG = torch.optim.Adam(netG.parameters(), lr=0.001)

In [ ]:
def train(netD, netG, batch_per_epoch, batch_size, latent_dim, epochs, loss_func, optD, optG):
    for epoch in range(epochs):
        for _ in range(batch_per_epoch):
            batchG = get_g_batch(batch_size, latent_dim)
            batchD = get_mix_batch(batch_size, latent_dim, netG)
            
            netD.train(True)
            netG.train(False)
            netD_step(netD, batchD, loss_func, optD)
            
            netD.train(False)
            netG.train(True)
            netG_step(netD, netG, batchG, loss_func, optG)
            
        # clear_output()
        plot_gen(netG, epoch)

### 1.1 Решение:

**Посмотреть что будет если подавать в качестве шума uniform распределение**

In [ ]:
# Code here
train(netD, netG, batch_per_epoch, batch_size, latent_dim, epochs, loss_func, optD, optG)

In [ ]:
netG = GenModel(latent_dim).cuda()
netD = DisModel().cuda()
loss_func = nn.BCELoss().cuda()
optD = torch.optim.Adam(netD.parameters(), lr=0.001)
optG = torch.optim.Adam(netG.parameters(), lr=0.001)
train(netD, netG, batch_per_epoch, batch_size, latent_dim, epochs, loss_func, optD, optG)

**Попробовать избавится от линейности на параболе (любыми известными методами)**

In [ ]:
# Code here
class GenModel_2(nn.Module):
    def __init__(self, latent_space):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_space, 50),
            nn.SELU(),
            nn.Linear(50, 50),
            nn.SELU(),
            nn.Linear(50,2))

    def forward(self, x):
        return self.model(x)

class DisModel_2(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 50),
            nn.SELU(),
            nn.Linear(50, 15),
            nn.SELU(),
            nn.Linear(15,1),
            nn.Sigmoid())
        
    def forward(self, x):
        return self.model(x)

In [ ]:
netG = GenModel_2(latent_dim).cuda()
netD = DisModel_2().cuda()
loss_func = nn.BCELoss().cuda()
optD = torch.optim.Adam(netD.parameters(), lr=0.001)
optG = torch.optim.Adam(netG.parameters(), lr=0.001)
train(netD, netG, batch_per_epoch, batch_size, latent_dim, 10, loss_func, optD, optG)

### 1.2 Решение:

**Сделать генерацию фигуры более сложной формы, например круга**

In [ ]:
# Code here
def get_mix_batch(batch_size, latent_dim, netG):
    types_of_points = []
    
    # Generate true pairs and true labels
    phi = torch.distributions.Uniform(-np.pi, np.pi).sample((batch_size,)).cuda()
    R=0.8
    true_pair = torch.vstack((R*torch.cos(phi), R*torch.sin(phi))).T.cuda()
    true_labels = torch.ones(size=(batch_size,)).unsqueeze(1).cuda()
    types_of_points.append(torch.hstack((true_pair, true_labels)))
    
    # Generate fake uniform pairs and fake labels
    if True:
        x_fake = torch.distributions.Uniform(-1, +1).sample((batch_size,)).cuda()
        y_fake = torch.distributions.Uniform(-1, +1).sample((batch_size,)).cuda()
        fake_pair = torch.vstack((x_fake, y_fake)).T
        fake_labels = torch.zeros(size=(batch_size,)).unsqueeze(1).cuda()
        types_of_points.append(torch.hstack((fake_pair, fake_labels)))

    # Generate points from generator and set labels as fake
    if True:
        gan_pair = netG(torch.randn(size=(batch_size, latent_dim)).cuda())
        #gan_pair = netG(torch.rand(size=(batch_size, latent_dim)).cuda())
        gan_labels = torch.zeros(size=(batch_size,)).unsqueeze(1).cuda()
        types_of_points.append(torch.hstack((gan_pair, gan_labels)))
    
    # Stack all types of points
    z = torch.vstack(types_of_points)
    # Shuffle
    z=z[torch.randperm(z.size()[0])]
    
    # Split back to samples and labels
    mixed_pairs = z[:, :2]
    mixed_labels = z[:, 2]
    return mixed_pairs, mixed_labels

def plot_gen(netG, epoch="Not provided"):
    Gin, _ = get_g_batch(1000, latent_dim)
    out = netG(Gin).cpu()
    plt.scatter(out.detach().numpy()[:, 0], out.detach().numpy()[:, 1], color="blue", s=1)
    plt.title(f'Generator points. End of epoch= {epoch+1}', fontsize=10)
    plt.axis([-1,1,-1,1])
    plt.show()

In [ ]:
netG = GenModel_2(latent_dim).cuda()
netD = DisModel_2().cuda()
loss_func = nn.BCELoss().cuda()
optD = torch.optim.Adam(netD.parameters(), lr=0.002)
optG = torch.optim.Adam(netG.parameters(), lr=0.001)
train(netD, netG, batch_per_epoch, batch_size, latent_dim, 10, loss_func, optD, optG)

**Гиф-анимация генерируемых точек в процессе обучения (опционально)**

In [ ]:
# Code here

### 1.3 Решение:

**Предобучить только дискриминатор (сделать его сильным критиком), а потом обучать только генератор**

In [ ]:
# Code here
netG = GenModel(latent_dim).cuda()
netD = DisModel().cuda()
loss_func = nn.BCELoss().cuda()
optD = torch.optim.Adam(netD.parameters(), lr=0.001)
optG = torch.optim.Adam(netG.parameters(), lr=0.001)

for epoch in range(10):
        for _ in range(batch_per_epoch):
            batchG = get_g_batch(batch_size, latent_dim)
            batchD = get_mix_batch(batch_size, latent_dim, netG)
            
            netD.train(True)
            netG.train(False)
            netD_step(netD, batchD, loss_func, optD)
            
            #netD.train(False)
            #netG.train(True)
            #netG_step(netD, netG, batchG, loss_func, optG)

In [ ]:
for epoch in range(10):
  for _ in range(batch_per_epoch):
    batchG = get_g_batch(batch_size, latent_dim)
    batchD = get_mix_batch(batch_size, latent_dim, netG)
            
    netD.train(False)
    netG.train(True)
    netG_step(netD, netG, batchG, loss_func, optG)
  plot_gen(netG, epoch)

## Задание 2. cGAN на датасете Fashion mnist (или MNIST)

Напишите полносвязный GAN с условием. Условием в данном случае будет являтся lablel (номер класса) цифры или вещи.

1. Сделайте эмбединг для лэйблов внутри модели
2. С помощью torch.cat добавьте этот эмбединг ко входу генератора и дискриминатора
3. Используйте такие параметры для генератора:
        self.model = nn.Sequential(
            nn.Linear(?, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )
        
4. Используйте такую архитектуру для дискриминатора:
        self.model = nn.Sequential(
            nn.Linear(?, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Code here
        self.label_emb = nn.Embedding(10, 10)
        
        self.model = nn.Sequential(
            nn.Linear(110, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )
        
    
    def forward(self, z, labels):
        
        # Code here
        z = z.view(z.size(0), 100)
        c = self.label_emb(labels)
        x = torch.cat([z, c], 1)
        
        out = self.model(x)
        return out.view(x.size(0), 28, 28)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.label_emb = nn.Embedding(10, 10)
        # Code here
        self.model = nn.Sequential(
            nn.Linear(794, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x, labels):
        
        # Code here
        x = x.view(x.size(0), 28*28)
        c = self.label_emb(labels)
        x = torch.cat([x, c], 1)
        out = self.model(x)
        return out.squeeze()

Код ниже желательно не изменять

In [ ]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5), std=(0.5))
])

dataset = FashionMNIST(root ='/content/',
                       train = True,
                       transform = transform, 
                       target_transform = None, 
                       download = True)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
def get_class_name(num):
    """Вспомогательная функция возвращающая название класса по его индексу
    num - численный индекс класса"""
    class_names = dataset.class_to_idx #{название класса : индекс класса}

    key_list = list(class_names.keys())
    val_list = list(class_names.values())
    
    # print key with val 100
    position = val_list.index(num)
    print(key_list[position])

In [ ]:
def generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion):
    g_optimizer.zero_grad()
    z = torch.randn(batch_size, 100).to(device)
    fake_labels = torch.LongTensor(np.random.randint(0, 10, batch_size)).to(device)
    fake_images = generator(z, fake_labels)
    validity = discriminator(fake_images, fake_labels)
    g_loss = criterion(validity, torch.ones(batch_size).to(device))
    g_loss.backward()
    g_optimizer.step()
    return g_loss

def discriminator_train_step(batch_size, discriminator, generator, d_optimizer, criterion, real_images, labels):
    d_optimizer.zero_grad()

    # train with real images
    real_validity = discriminator(real_images, labels)
    real_loss = criterion(real_validity, torch.ones(batch_size).to(device))
    # train with fake images
    z = torch.randn(batch_size, 100).to(device)
    fake_labels = torch.LongTensor(np.random.randint(0, 10, batch_size)).to(device)
    fake_images = generator(z, fake_labels)
    fake_validity = discriminator(fake_images, fake_labels)
    fake_loss = criterion(fake_validity, torch.zeros(batch_size).to(device))

    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss

In [ ]:
def show_gen_res(generator):
    z = torch.randn(9, 100).to(device)
    labels = torch.LongTensor(np.arange(9)).to(device)

    sample_images = generator(z, labels).unsqueeze(1).data.cpu()

    grid = make_grid(sample_images, nrow=3, normalize=True).permute(1,2,0).numpy()
    plt.imshow(grid)
    plt.show()

In [ ]:
betas = (0.5, 0.999)
device='cuda'
generator = Generator().to(device)
discriminator = Discriminator().cuda()
lr = 1e-4

criterion = nn.BCELoss().cuda()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=betas)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr, betas=betas)

In [ ]:
num_epochs = 10
n_critic = 5
display_step = 300

for epoch in range(num_epochs):
    show_gen_res(generator)
    print('Starting epoch {}...'.format(epoch))
    for i, (images, labels) in enumerate(data_loader):
        real_images = images.to(device)
        labels = labels.to(device)
        generator.train()
        batch_size = real_images.size(0)
        d_loss = discriminator_train_step(len(real_images), discriminator,
                                          generator, d_optimizer, criterion,
                                          real_images, labels)
        

        g_loss = generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion)

    generator.eval()
    print('g_loss: {}, d_loss: {}'.format(g_loss, d_loss))

## Задание 3. cDCGAN (Дополнительное)
* Решить предыдущую задачу при помощи развёрток
* Основная проблема в том, что тут очень тяжело подобрать правильные коэффициенты.
* Эту задачу можно решать абсолютно любым способом (вполть до замены всего кода), единственные условия: сеть должна быть построена по архитектуре cDCGAN и генерировать на датасте Fashion mnist

Ниже пример неправильно подобранных параметров генератора. (Дискриминатор используется такой же как выше). Даже если учить эту сеть очень долго, результаты получаются очень далёкими от хороших.

In [ ]:
class Generator(nn.Module):
    def __init__(self, d=128):
        super().__init__()
        
        self.label_emb = nn.Embedding(10, 10)
        
        self.deconv1_1 = nn.ConvTranspose2d(100, d*2, 4, 1, 0)
        self.deconv1_1_bn = nn.BatchNorm2d(d*2)
        self.deconv1_2 = nn.ConvTranspose2d(10, d*2, 4, 1, 0)
        self.deconv1_2_bn = nn.BatchNorm2d(d*2)
        self.deconv2 = nn.ConvTranspose2d(d*4, d*2, 4, 2, 1)
        self.deconv2_bn = nn.BatchNorm2d(d*2)
        self.deconv3 = nn.ConvTranspose2d(d*2, d, 4, 2, 1)
        self.deconv3_bn = nn.BatchNorm2d(d)
        
        self.deconv4 = nn.ConvTranspose2d(d, out_channels=1, kernel_size=2, stride=2, padding=2)
        
    
    def forward(self, z, labels):
        z = z.view(z.size(0), 100, 1, 1)
        c=self.label_emb(labels)
        c = c.view(c.shape[0],10,1,1)
    
        x=self.deconv1_1(z)
        x = F.relu(self.deconv1_1_bn(x))
        y = F.relu(self.deconv1_2_bn(self.deconv1_2(c)))
        x = torch.cat([x, y], 1)
        x = F.relu(self.deconv2_bn(self.deconv2(x)))
        x = F.relu(self.deconv3_bn(self.deconv3(x)))
        
        x = F.tanh(self.deconv4(x))
        
        return x.view(x.size(0), 28, 28)

In [ ]:
generator = Generator().to(device)
discriminator = Discriminator().cuda()
lr = 1e-4

criterion = nn.BCELoss().cuda()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=betas)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr, betas=betas)

In [ ]:
num_epochs = 20
n_critic = 5
display_step = 300

for epoch in range(num_epochs):
    show_gen_res(generator)
    print('Starting epoch {}...'.format(epoch))
    for i, (images, labels) in enumerate(data_loader):
        real_images = images.to(device)
        labels = labels.to(device)
        generator.train()
        batch_size = real_images.size(0)
        d_loss = discriminator_train_step(len(real_images), discriminator,
                                          generator, d_optimizer, criterion,
                                          real_images, labels)
        

        g_loss = generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion)

    generator.eval()
    print('g_loss: {}, d_loss: {}'.format(g_loss, d_loss))